In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
!wget -qq https://www.dropbox.com/s/pan6mutc5xj5kj0/trainPart1.zip

In [3]:
!unzip -qq trainPart1.zip

In [4]:
!ls train

Sample001  Sample010  Sample019  Sample028  Sample037  Sample046  Sample055
Sample002  Sample011  Sample020  Sample029  Sample038  Sample047  Sample056
Sample003  Sample012  Sample021  Sample030  Sample039  Sample048  Sample057
Sample004  Sample013  Sample022  Sample031  Sample040  Sample049  Sample058
Sample005  Sample014  Sample023  Sample032  Sample041  Sample050  Sample059
Sample006  Sample015  Sample024  Sample033  Sample042  Sample051  Sample060
Sample007  Sample016  Sample025  Sample034  Sample043  Sample052  Sample061
Sample008  Sample017  Sample026  Sample035  Sample044  Sample053  Sample062
Sample009  Sample018  Sample027  Sample036  Sample045  Sample054


In [5]:
# Checking for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
transformer = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5])
])

In [7]:
import os
import random
import shutil

train_path = '/content/train'
validation_path = '/content/validation'

if not os.path.exists(validation_path):
  os.makedirs(validation_path)

validation_percentage = 10

def move_random_image(src_dir, dest_dir, percentage):
  if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

  image_files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
  print(image_files)

  num_images_to_move = int(len(image_files) * (percentage/100))

  # Randomly select 'num_images_to_move' images
  images_to_move = random.sample(image_files, num_images_to_move)

  for image in images_to_move:
    src = os.path.join(src_dir, image)
    dest = os.path.join(dest_dir, image)
    shutil.move(src, dest)

for class_folder in os.listdir(train_path):
  class_train_dir = os.path.join(train_path, class_folder)
  class_validation_dir = os.path.join(validation_path, class_folder)
  move_random_image(class_train_dir, class_validation_dir, validation_percentage)


['img021-027.png', 'img021-005.png', 'img021-044.png', 'img021-023.png', 'img021-008.png', 'img021-053.png', 'img021-026.png', 'img021-019.png', 'img021-021.png', 'img021-036.png', 'img021-034.png', 'img021-001.png', 'img021-024.png', 'img021-046.png', 'img021-004.png', 'img021-054.png', 'img021-042.png', 'img021-052.png', 'img021-010.png', 'img021-035.png', 'img021-028.png', 'img021-003.png', 'img021-038.png', 'img021-022.png', 'img021-032.png', 'img021-029.png', 'img021-031.png', 'img021-009.png', 'img021-033.png', 'img021-040.png', 'img021-002.png', 'img021-047.png', 'img021-018.png', 'img021-016.png', 'img021-011.png', 'img021-037.png', 'img021-048.png', 'img021-051.png', 'img021-041.png', 'img021-050.png']
['img002-020.png', 'img002-026.png', 'img002-043.png', 'img002-040.png', 'img002-013.png', 'img002-045.png', 'img002-007.png', 'img002-018.png', 'img002-048.png', 'img002-042.png', 'img002-019.png', 'img002-008.png', 'img002-001.png', 'img002-006.png', 'img002-032.png', 'img002-

In [8]:
# DataLoader
BATCH_SIZE = 32

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size = BATCH_SIZE,
    shuffle=True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(validation_path, transform=transformer),
    batch_size = BATCH_SIZE,
    shuffle=True
)

In [9]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
num_classes = len(classes)
classes

['Sample001',
 'Sample002',
 'Sample003',
 'Sample004',
 'Sample005',
 'Sample006',
 'Sample007',
 'Sample008',
 'Sample009',
 'Sample010',
 'Sample011',
 'Sample012',
 'Sample013',
 'Sample014',
 'Sample015',
 'Sample016',
 'Sample017',
 'Sample018',
 'Sample019',
 'Sample020',
 'Sample021',
 'Sample022',
 'Sample023',
 'Sample024',
 'Sample025',
 'Sample026',
 'Sample027',
 'Sample028',
 'Sample029',
 'Sample030',
 'Sample031',
 'Sample032',
 'Sample033',
 'Sample034',
 'Sample035',
 'Sample036',
 'Sample037',
 'Sample038',
 'Sample039',
 'Sample040',
 'Sample041',
 'Sample042',
 'Sample043',
 'Sample044',
 'Sample045',
 'Sample046',
 'Sample047',
 'Sample048',
 'Sample049',
 'Sample050',
 'Sample051',
 'Sample052',
 'Sample053',
 'Sample054',
 'Sample055',
 'Sample056',
 'Sample057',
 'Sample058',
 'Sample059',
 'Sample060',
 'Sample061',
 'Sample062']

In [10]:
#CNN Network
class ConvNet(nn.Module):
  def __init__(self,num_classes = 62):
    super(ConvNet, self).__init__()
    # Output size after convolution filter
    # ((w-f+2P)/s) + 1

    # Input shape = (32, 3, 150, 150)

    self.conv1=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
    # Shapes (32,12,150,150)
    self.bn1=nn.BatchNorm2d(num_features=12)
    # Shapes (32,12,150,150)
    self.relu1 = nn.ReLU()
    # Shapes (32,12,150,150)

    self.pool = nn.MaxPool2d(kernel_size=2)
    # Reduce the image size be factor of 2
    # Shape=(32, 12, 75, 75)

    self.conv2=nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
    # Shapes (32,20,75,75)
    self.relu2 = nn.ReLU()
    # Shapes (32,20,75,75)

    self.conv3=nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
    # Shapes (32,32,75,75)
    self.bn3=nn.BatchNorm2d(num_features=32)
    # Shapes (32,32,75,75)
    self.relu3 = nn.ReLU()
    # Shapes (32,32,75,75)

    self.fc=nn.Linear(in_features=32*75*75, out_features=num_classes)

    # Feed forward function

  def forward(self, input):
    output=self.conv1(input)
    output=self.bn1(output)
    output=self.relu1(output)

    output=self.pool(output)

    output=self.conv2(output)
    output=self.relu2(output)

    output=self.conv3(output)
    output=self.bn3(output)
    output=self.relu3(output)

      # Above output will be in metrix form, with shape (32, 32, 75, 75)

    output = output.view(-1, 32*75*75)

    output = self.fc(output)
    return output


In [11]:

model=ConvNet(num_classes=62).to(device)

In [12]:
# Optimizer and loss function
optimizer=Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [13]:
num_epochs = 10

In [14]:
train_count = 0
for cls in classes:
  train_count += len(os.listdir(os.path.join(train_path, cls)))

test_count = 0
for cls in classes:
  test_count += len(os.listdir(os.path.join(validation_path, cls)))

train_count, test_count

(2232, 248)

In [15]:
# Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):

  # Evaluating and training on training dataset
  model.train()
  train_accuracy=0.0
  train_loss=0.0

  for i, (images, labels) in enumerate(train_loader):
    if torch.cuda.is_available():
      images=Variable(images.cuda())
      labels=Variable(labels.cuda())

    optimizer.zero_grad()

    outputs = model(images)
    loss=loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.cpu().data*images.size(0)
    _,prediction=torch.max(outputs.data, 1)

    train_accuracy+=int(torch.sum(prediction==labels.data).item())

  train_accuracy=train_accuracy/train_count
  train_loss=train_loss/train_count

  # Evaluating on testing dataset
  model.eval()

  test_accuracy = 0.0
  for i, (images, labels) in enumerate(test_loader):
    if torch.cuda.is_available():
      images=Variable(images.cuda())
      labels=Variable(labels.cuda())

    outputs = model(images)
    _,prediction= torch.max(outputs.data, 1)
    test_accuracy += int(torch.sum(prediction==labels.data).item())

  test_accuracy=test_accuracy/test_count

  print(f"Epoch: {epoch} | Train loss: {train_loss} | Train Accuracy: {train_accuracy} | Test Accuracy: {test_accuracy}")

  # Save the model
  if test_accuracy > best_accuracy:
    torch.save(model.state_dict(), 'best_checkpoint.model')
    best_accuracy=test_accuracy

Epoch: 0 | Train loss: 35.65538787841797 | Train Accuracy: 0.08646953405017921 | Test Accuracy: 0.12096774193548387
Epoch: 1 | Train loss: 5.978155612945557 | Train Accuracy: 0.4422043010752688 | Test Accuracy: 0.07661290322580645
Epoch: 2 | Train loss: 1.6344293355941772 | Train Accuracy: 0.7531362007168458 | Test Accuracy: 0.22983870967741934
Epoch: 3 | Train loss: 0.5966605544090271 | Train Accuracy: 0.8911290322580645 | Test Accuracy: 0.27419354838709675
Epoch: 4 | Train loss: 0.3079034388065338 | Train Accuracy: 0.9363799283154122 | Test Accuracy: 0.1975806451612903
Epoch: 5 | Train loss: 0.15343749523162842 | Train Accuracy: 0.9623655913978495 | Test Accuracy: 0.2903225806451613
Epoch: 6 | Train loss: 0.15127356350421906 | Train Accuracy: 0.9672939068100358 | Test Accuracy: 0.23387096774193547
Epoch: 7 | Train loss: 0.12457588315010071 | Train Accuracy: 0.9762544802867383 | Test Accuracy: 0.29435483870967744
Epoch: 8 | Train loss: 0.057334449142217636 | Train Accuracy: 0.98790322

In [16]:
checkpoint = torch.load('/content/best_checkpoint.model')
model = ConvNet(num_classes = 62)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=62, bias=True)
)

In [17]:
transformer = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5])
])

In [18]:
# Prediction function
from PIL import Image

def prediction(img_path, transformer):

  image = Image.open(img_path)

  image_tensor=transformer(image).float()

  image_tensor=image_tensor.unsqueeze_(0)
  if torch.cuda.is_available():
    image_tensor.cuda()

  input=Variable(image_tensor)

  output=model(input)

  index = output.data.numpy().argmax()

  pred=classes[index]

  return pred

In [19]:
images_path = []
for cls in classes:
  # images_path.append(os.path.join(validation_path,cls))
  dir_path = os.path.join(validation_path,cls)
  for images in os.listdir(dir_path):
    images_path.append(os.path.join(dir_path, images))

images_path

['/content/validation/Sample001/img001-024.png',
 '/content/validation/Sample001/img001-001.png',
 '/content/validation/Sample001/img001-027.png',
 '/content/validation/Sample001/img001-044.png',
 '/content/validation/Sample002/img002-045.png',
 '/content/validation/Sample002/img002-001.png',
 '/content/validation/Sample002/img002-024.png',
 '/content/validation/Sample002/img002-053.png',
 '/content/validation/Sample003/img003-006.png',
 '/content/validation/Sample003/img003-047.png',
 '/content/validation/Sample003/img003-050.png',
 '/content/validation/Sample003/img003-041.png',
 '/content/validation/Sample004/img004-028.png',
 '/content/validation/Sample004/img004-038.png',
 '/content/validation/Sample004/img004-036.png',
 '/content/validation/Sample004/img004-029.png',
 '/content/validation/Sample005/img005-052.png',
 '/content/validation/Sample005/img005-024.png',
 '/content/validation/Sample005/img005-001.png',
 '/content/validation/Sample005/img005-020.png',
 '/content/validatio

In [20]:
for i in images_path:
  print(prediction(i, transformer))

Sample027
Sample014
Sample059
Sample013
Sample022
Sample011
Sample048
Sample002
Sample062
Sample011
Sample003
Sample003
Sample004
Sample010
Sample010
Sample017
Sample010
Sample038
Sample033
Sample018
Sample010
Sample051
Sample015
Sample017
Sample029
Sample013
Sample007
Sample017
Sample010
Sample010
Sample010
Sample008
Sample009
Sample003
Sample062
Sample009
Sample003
Sample010
Sample002
Sample053
Sample005
Sample011
Sample011
Sample042
Sample003
Sample003
Sample012
Sample038
Sample013
Sample013
Sample013
Sample007
Sample026
Sample014
Sample014
Sample026
Sample015
Sample015
Sample036
Sample062
Sample016
Sample026
Sample016
Sample020
Sample017
Sample010
Sample012
Sample017
Sample018
Sample056
Sample036
Sample018
Sample010
Sample014
Sample002
Sample002
Sample011
Sample020
Sample046
Sample046
Sample047
Sample023
Sample016
Sample021
Sample036
Sample038
Sample047
Sample041
Sample023
Sample023
Sample024
Sample023
Sample032
Sample024
Sample031
Sample024
Sample026
Sample016
Sample025
Sample031
